# Import

In [16]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime as dt
from helpful_functions import *
from exporation.plotting import CandlePlot
import numpy as np
import plotly.graph_objects as go
import math

# Config

In [51]:
mt5.initialize()
LOGIN = 270397308
PASSWORD = "@Pass1234"
SERVER = "Exness-MT5Trial17"

mt5.login(LOGIN, PASSWORD, SERVER)

PAIR = "BTCUSD"
RISK_FREE_RATE = 0.00

TIMEFRAME = mt5.TIMEFRAME_H4
START_DATE = dt(2020, 1, 1)
END_DATE = dt(2025, 10, 30)

INITIAL_CAPITAL = 10000
RISK_PER_TRADE = 0.02
COMMISSION_PER_LOT = 2 # two way USD commission per lot

# Function

In [18]:
def get_data_from_mt5(pair, timeframe, start_date, end_date):
    avai_symbol_names = [symbol.name for symbol in mt5.symbols_get()]
    if pair not in avai_symbol_names:
        print(f'{pair} is not available in MT5 terminal')
    else:
        df = pd.DataFrame(mt5.copy_rates_range(pair, timeframe, start_date, end_date))
        df.time = pd.to_datetime(df.time, unit="s")
        print(f"Getting data of {pair}_{timeframe} successfully ")
    return df

def create_excel_from_mt5(pair, timeframe, start_date, end_date, file_path):
    df = get_data_from_mt5(pair, timeframe, start_date, end_date)
    
    start_year = start_date.year
    end_year = end_date.year
    timeframe_str = {mt5.TIMEFRAME_M1: "M1", mt5.TIMEFRAME_M5: "M5", mt5.TIMEFRAME_M15: "M15", mt5.TIMEFRAME_M30: "M30", mt5.TIMEFRAME_H1: "H1", mt5.TIMEFRAME_H4: "H4"}[timeframe]
    filename = f"{pair}_{timeframe_str}_{start_year}_{end_year}.xlsx"
    file_full_path = f"{file_path}\\{filename}"
    
    df.to_excel(file_full_path, index=False)
    print(f"Data saved in {file_full_path}")
    
    return df


def get_digits_number(pair: str):
    if not mt5.initialize():
        raise RuntimeError("MT5 is not initialized.")
    
    else:
        symbols = mt5.symbols_get()
        digits_dict = {symbol.name: symbol.digits for symbol in symbols}
        return digits_dict.get(pair, None)

def add_bid_ask_columns(pair : str, df: pd.DataFrame):
    bid_ask_df = df.copy()
    
    digit = get_digits_number(pair)
    if digit is None:
        raise ValueError(f"Could not retrieve digits for pair: {pair}")
     
    bid_ask_df['real_spread'] = bid_ask_df['spread'] * (10 ** -digit)

    for col in ["open", "high", "low", "close"]:
        bid_ask_df[f"bid_{col[0]}"] = bid_ask_df[col]
        bid_ask_df[f"ask_{col[0]}"] = bid_ask_df[col] + bid_ask_df['real_spread']
    
    bid_ask_df.drop(columns=["open", "high", "low", "close"], inplace=True)

    return bid_ask_df

def donchian_breakout_channel(df, lookback = 50, close_col_name='bid_c', spread_col='real_spread'):
    """ 
    Generates Donchian Channel breakout signals and stop-loss levels
    """
    
    df['donchian_high'] = df[close_col_name].rolling(window=lookback -1).max().shift(1)
    df['donchian_low'] = df[close_col_name].rolling(window=lookback -1).min().shift(1)

    df['signal'] = 0
    df.loc[df[close_col_name] > df['donchian_high'], 'signal'] = 1
    df.loc[df[close_col_name] < df['donchian_low'], 'signal'] = -1
    df['signal'] = df['signal'].ffill().fillna(0).astype(int)

    # Store entry value
    s = df['signal']
    df['entry'] = np.where(( s!= 0 ) & (s != s.shift(1)), s, 0).astype(int)

    # Store current position
    df['position'] = df['entry'].replace(0, np.nan).ffill().fillna(0).astype(int)

    # Add Stop Loss
    spread_tick = df[spread_col]
    df['sl_buy'] = df['donchian_low'] - spread_tick
    df['sl_sell'] = df['donchian_high'] + spread_tick

    return df


def calculate_lot_size(pair, entry_price, stop_loss, capital, risk_pct, order_type):
    """ 
    Calculate lot size based on risk percentage. If min_volume leads to a loss greater than list_pct return 0
    """
    
    risk_money = capital * risk_pct
    info = mt5.symbol_info(pair)

    vol_step = float(info.volume_step)
    min_vol = float(info.volume_min)
    max_vol = float(info.volume_max)

    # Calculate loss per 1 lot 
    order_type_mt5 = mt5.ORDER_TYPE_BUY if order_type == "BUY" else mt5.ORDER_TYPE_SELL
    loss_per_1lot = abs(mt5.order_calc_profit(order_type_mt5, pair, 1, entry_price, stop_loss))
    if loss_per_1lot <= 0:
        return 0
    
    # Calculate final lot size
    raw_lot = risk_money / loss_per_1lot
    raw_lot = max(min_vol, min(max_vol, raw_lot))

    steps = (raw_lot - min_vol) / vol_step
    steps_floor = math.floor(steps)
    lot = round(min_vol + steps_floor * vol_step, 2)

    # Check if min_volume leads to a loss greater than risk_pct
    potential_loss = abs(mt5.order_calc_profit(order_type_mt5, pair, lot, entry_price, stop_loss))
    if potential_loss > risk_money:
        return 0
    
    return lot

def backtest_donchian_trades(pair, df, capital, risk_pct, commission):
    """ 
    Backtest donchian breakout strategy with given DataFrame and return a DataFrame of trade results 
    """

    trade_log = []

    for i, row in df.iterrows():
        if row['entry'] != 0:
            side = "BUY" if row['entry'] == 1 else "SELL"
            if side == "BUY": 
                entry_price = row['ask_c'] 
                stop_loss = row['sl_buy']
            else:
                entry_price = row['bid_c'] 
                stop_loss = row['sl_sell']

            # Calculate lot size
            lot = calculate_lot_size(pair, entry_price, stop_loss, capital, risk_pct, side)
            if lot == 0:
                continue

            # Find exit price and calculate PnL
            exit_row = None
            for j in range(i+1, len(df)):
                next_bar = df.iloc[j]
                # opposite entry -> exit trade
                if next_bar['entry'] == -row['entry']:
                    exit_row = next_bar
                    exit_reason = "Reverse_signal"
                    break
                # hit stop loss
                if side == "BUY" and next_bar['bid_l'] <= stop_loss:
                    exit_row = next_bar 
                    exit_reason = "SL"    
                    break
                if side == "SELL" and next_bar['ask_h'] >= stop_loss:
                    exit_row = next_bar
                    exit_reason = "SL"
                    break

            # If trade still open at the end of data, close at last bar
            if exit_row is None:
                exit_row = df.iloc[-1]
                exit_reason = "End of Data"

            if exit_reason == "SL":
                exit_price = stop_loss
            else:
                exit_price = exit_row['bid_c'] if side == 'BUY' else exit_row['ask_c']

            # Calculate PnL
            order_type_mt5 = mt5.ORDER_TYPE_BUY if side == "BUY" else mt5.ORDER_TYPE_SELL
            profit_bc = mt5.order_calc_profit(order_type_mt5, pair, lot, entry_price, exit_price)
            total_commission = commission * lot
            profit_ac = profit_bc -total_commission

            capital += profit_ac

            trade_log.append({
                "symbol": pair,
                "entry_time": row['time'],
                "exit_time": exit_row['time'],
                "exit_reason": exit_reason,
                "side": side,
                "lot": lot,
                "entry_price": entry_price,
                "exit_price": exit_price,
                "profit_bc": profit_bc,
                "commission": total_commission,
                "profit_ac": profit_ac,
                "acc_balance": capital
            })

    trade_df = pd.DataFrame(trade_log)
    return trade_df

def donchian_breakout_channel_v2(df, lookback=50, close_col_name='bid_c', spread_col='real_spread'):
    """
    Donchian breakout v2: only open 1 position at a time
    """
    df = df.copy()

    dh = df[close_col_name].rolling(window=lookback-1).max().shift(1)
    dl = df[close_col_name].rolling(window=lookback-1).min().shift(1)
    df['donchian_high'] = dh
    df['donchian_low']  = dl

    # 2) Tín hiệu thô của NẾN HIỆN TẠI (không ffill)
    #    +1: close > dh, -1: close < dl, 0: còn lại
    sig_raw = np.where(df[close_col_name] > df['donchian_high'],  1,
               np.where(df[close_col_name] < df['donchian_low'],  -1, 0))
    df['signal_raw'] = sig_raw

    # 3) Sinh entry & position theo state machine để tránh vào chồng lệnh
    entry = np.zeros(len(df), dtype=int)
    position = np.zeros(len(df), dtype=int)  # -1/0/1

    for i in range(len(df)):
        s = sig_raw[i]          # breakout ở nến hiện tại (có thể 0)
        pos_prev = position[i-1] if i > 0 else 0

        if pos_prev == 0:
            # đang flat: chỉ vào nếu có breakout (±1)
            if s != 0:
                entry[i] = s
                position[i] = s
            else:
                position[i] = 0
        else:
            # đang có vị thế: chỉ vào khi breakout đảo chiều
            if s != 0 and s == -pos_prev:
                entry[i] = s
                position[i] = s    # đảo chiều luôn
            else:
                position[i] = pos_prev  # giữ nguyên, KHÔNG vào thêm cùng chiều

    df['entry'] = entry.astype(int)
    df['position'] = position.astype(int)

    spread_tick = df[spread_col].astype(float)
    df['sl_buy']  = df['donchian_low']  - spread_tick
    df['sl_sell'] = df['donchian_high'] + spread_tick

    return df


# Main

In [19]:
btc_h1_raw = get_data_from_mt5(PAIR, TIMEFRAME, START_DATE, END_DATE)
btc_h1 = add_bid_ask_columns(PAIR, btc_h1_raw)
btc_h1_signal_v1 = donchian_breakout_channel(btc_h1, lookback=50, close_col_name='bid_c', spread_col='real_spread')
trade_results_v1 = backtest_donchian_trades(PAIR, btc_h1_signal_v1, INITIAL_CAPITAL, RISK_PER_TRADE, COMMISSION_PER_LOT)
trade_results_v1

Getting data of BTCUSD_16388 successfully 


,symbol,entry_time,exit_time,exit_reason,side,lot,entry_price,exit_price,profit_bc,commission,profit_ac,acc_balance
0,BTCUSD,2020-01-14 00:00:00,2020-01-23 08:00:00,Reverse_signal,BUY,0.17,8460.22,8389.45,-12.03,0.34,-12.37,9987.63
1,BTCUSD,2020-01-15 20:00:00,2020-01-23 08:00:00,Reverse_signal,BUY,0.18,8808.63,8389.45,-75.45,0.36,-75.81,9911.82
2,BTCUSD,2020-01-17 04:00:00,2020-01-23 08:00:00,Reverse_signal,BUY,0.16,8914.10,8389.45,-83.95,0.32,-84.27,9827.55
3,BTCUSD,2020-01-18 00:00:00,2020-01-23 08:00:00,Reverse_signal,BUY,0.16,8919.53,8389.45,-84.81,0.32,-85.13,9742.42
4,BTCUSD,2020-01-19 00:00:00,2020-01-23 08:00:00,Reverse_signal,BUY,0.17,9139.34,8389.45,-127.48,0.34,-127.82,9614.60
...,...,...,...,...,...,...,...,...,...,...,...,...
841,BTCUSD,2025-10-10 16:00:00,2025-10-26 12:00:00,Reverse_signal,SELL,2.33,116674.63,113701.08,6928.37,4.66,6923.71,1035167.41
842,BTCUSD,2025-10-16 12:00:00,2025-10-26 12:00:00,Reverse_signal,SELL,1.39,108584.45,113701.08,-7112.11,2.78,-7114.89,1028052.52
843,BTCUSD,2025-10-17 04:00:00,2025-10-26 12:00:00,Reverse_signal,SELL,1.19,105618.27,113701.08,-9618.55,2.38,-9620.93,1018431.59
844,BTCUSD,2025-10-26 12:00:00,2025-10-29 16:00:00,End of Data,BUY,2.94,113701.08,110655.40,-8954.30,5.88,-8960.18,1009471.41


In [20]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=trade_results_v1['entry_time'],
    y=trade_results_v1['acc_balance'],
    mode='lines',
    name='Account Balance'
))

fig.update_layout(
    title='Account Balance Over Time',
    xaxis_title='Entry Time',
    yaxis_title='Account Balance',
    template='plotly_dark'
)

fig.show()

In [21]:
btc_h1_signal_v2 = donchian_breakout_channel_v2(btc_h1, lookback=50, close_col_name='bid_c', spread_col='real_spread')
trade_results_v2 = backtest_donchian_trades(PAIR, btc_h1_signal_v2, INITIAL_CAPITAL, RISK_PER_TRADE, COMMISSION_PER_LOT)
trade_results_v2

,symbol,entry_time,exit_time,exit_reason,side,lot,entry_price,exit_price,profit_bc,commission,profit_ac,acc_balance
0,BTCUSD,2020-01-14 00:00:00,2020-01-23 08:00:00,Reverse_signal,BUY,0.17,8460.22,8389.45,-12.03,0.34,-12.37,9987.63
1,BTCUSD,2020-01-23 08:00:00,2020-01-27 12:00:00,Reverse_signal,SELL,0.26,8389.45,8759.47,-96.20,0.52,-96.72,9890.91
2,BTCUSD,2020-01-27 12:00:00,2020-02-17 08:00:00,Reverse_signal,BUY,0.40,8759.47,9636.65,350.87,0.80,350.07,10240.98
3,BTCUSD,2020-02-17 08:00:00,2020-03-05 16:00:00,Reverse_signal,SELL,0.27,9636.65,9172.76,125.25,0.54,124.71,10365.69
4,BTCUSD,2020-03-05 16:00:00,2020-03-08 12:00:00,Reverse_signal,BUY,0.31,9172.76,8411.50,-235.99,0.62,-236.61,10129.08
...,...,...,...,...,...,...,...,...,...,...,...,...
146,BTCUSD,2025-09-09 04:00:00,2025-09-22 00:00:00,Reverse_signal,BUY,0.08,113001.20,114716.75,137.24,0.16,137.08,22519.25
147,BTCUSD,2025-09-22 00:00:00,2025-10-01 04:00:00,Reverse_signal,SELL,0.15,114716.75,114578.91,20.67,0.30,20.37,22539.62
148,BTCUSD,2025-10-01 04:00:00,2025-10-10 16:00:00,Reverse_signal,BUY,0.08,114578.91,116674.63,167.66,0.16,167.50,22707.12
149,BTCUSD,2025-10-10 16:00:00,2025-10-26 12:00:00,Reverse_signal,SELL,0.05,116674.63,113701.08,148.68,0.10,148.58,22855.70


In [22]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=trade_results_v2['entry_time'],
    y=trade_results_v2['acc_balance'],
    mode='lines',
    name='Account Balance'
))

fig2.update_layout(
    title='Account Balance Over Time',
    xaxis_title='Entry Time',
    yaxis_title='Account Balance',
    template='plotly_dark'
)

fig2.show()

In [ ]:
def acc_balance_from_signals(signals_df, trade_df, initial_capital = INITIAL_CAPITAL,
                             time_col ='time', exit_col='exit_time'):
    """ 
    Return a Series of account balance over time based on signals and trade results.
    
    signal_df: df from donchian_breakout_channel() or strategy signal function
    trade_df: df from backtest_donchian_trades() or backtest function
    """

    sig = signals_df[[time_col]].copy()
    sig[time_col] = pd.to_datetime(sig[time_col])
    sig = sig.set_index(time_col).sort_index()

    exits = trade_df[[exit_col, 'acc_balance']].copy()
    exits[exit_col] = pd.to_datetime(exits[exit_col])
    exits = exits.set_index(exit_col).sort_index()

    merged = sig.join(exits[['acc_balance']], how='left')

    if len(merged) == 0:
        return pd.Series([], name='acc_balance')
    
    merged.loc[merged.index[0], 'acc_balance'] = float(initial_capital)
    merged['acc_balance'] = merged['acc_balance'].ffill().fillna(float(initial_capital))

    return merged['acc_balance']


def to_balance_daily(balance_series: pd.Series):
    """ 
    Convert balance series to daily frequency by resampling and ffill
    """
    # group by day, take the last value of each day then ffill
    
    daily_balance = balance_series.resample('D').last().ffill()
    daily_balance.name = 'balance_daily'
    return daily_balance


def drawdown_stats(balance_series, return_dd_series=False):
    """ 
    Calculate drawdown statistics from balance series and return a dictionary of max DD, avg DD, max DD duration, avg DD duration
    If return_dd_series is True return the drawdown stats dictionary & drawdown series for plotting the DD over time    
    """

    peak = balance_series.cummax()
    dd = balance_series/peak - 1.0
    dd_pct = dd * 100.0

    # Calculate durations of drawdowns
    durations, cur = [], 0
    for is_dd in (dd < 0).tolist():
        if is_dd:
            cur += 1
        else:
            if cur > 0:
                durations.append(cur)
            cur = 0
    if cur > 0:
        durations.append(cur)

    dd_stats = {
        'max_dd_pct': float(dd_pct.min()) if dd_pct.any() else 0.0,
        'avg_dd_pct': float(dd_pct[dd_pct < 0].mean()) if (dd_pct < 0).any() else 0.0,
        'max_dd_duration_days': int(max(durations)) if durations else 0,
        'avg_dd_duration_days': float(np.mean(durations)) if durations else 0.0
    }
    if return_dd_series:
        return dd_stats, dd_pct
    else:
        return dd_stats
    

def sharpe_sortino_from_balance(balance_series, rf_daily=RISK_FREE_RATE):
    """ 
    Calculate Sharpe and Sortino ratios from balance series
    """ 
    balance_daily = to_balance_daily(balance_series)
    rets = balance_daily.pct_change().dropna()
    if len(rets) == 0 or rets.std(ddof=1) == 0:
        return 0.0, 0.0
    
    excess_rets = rets - rf_daily
    sharpe = float(excess_rets.mean()/rets.std(ddof=1) * np.sqrt(252))

    downside_rets = rets[rets < 0]
    if len(downside_rets) == 0 or downside_rets.std(ddof=1) == 0:
        sortino = float('inf') # No downside volatility => infinite sortino
    else:
        sortino = float(excess_rets.mean()/downside_rets.std(ddof=1) * np.sqrt(252))
    
    return sharpe, sortino

In [ ]:
def performance_report(signals_df, trade_df, initial_capital=INITIAL_CAPITAL,
                       start_date=None, end_date=None, time_col='time'):
    """ 
    Build a performance report in form of DataFrame from signals and trade results:
    - signal_df: DataFrame from strategy signal function
    - trade_df : DataFrame from backtest results function
    - initial_capital: starting balance
    - start_date, end_date: optional override for date range
    """
    
    # Full time series of account balance
    balance_series = acc_balance_from_signals(signals_df, trade_df, initial_capital, time_col=time_col)
    if balance_series.empty:
        raise ValueError("Balance series is empty")
    
    # Convert to daily balance series
    balance_daily = to_balance_daily(balance_series)

    start = pd.to_datetime(start_date) if start_date else balance_daily.index.min()
    end = pd.to_datetime(end_date) if end_date else balance_daily.index.max()
    duration_days = int((end-start).days)
    years = max(duration_days/365.25, 1e-9)
    months = max(duration_days/30.44, 1e-9)

    # Compute metrics
    # Return metrics
    balance_final = float(balance_daily.iloc[-1])
    balance_peak = float(balance_daily.max())
    total_return = (balance_final/initial_capital -1) * 100.0
    annualized_return = ((balance_final/initial_capital) ** (1/years) -1) * 100.0
    monthly_return = ((balance_final/initial_capital) ** (1/months) -1) * 100.0

    # Risk metrics
    sharpe, sortino = sharpe_sortino_from_balance(balance_series)
    dd_stats = drawdown_stats(balance_series)

    # Trade-level metrics
    n = len(trade_df)

    cap_before = trade_df['acc_balance'] - trade_df['profit_ac']
    trade_pct_ret = (trade_df['profit_ac'] / cap_before.replace(0, np.nan)) * 100.0
    trade_pct_ret = trade_pct_ret.replace([np.inf, -np.inf], np.nan)

    # Winrate, % returns per trade
    winrate = float((trade_df['profit_ac'] > 0).sum() / n * 100.0)
    best_trade = float(np.nanmax(trade_pct_ret))
    worst_trade = float(np.nanmin(trade_pct_ret))
    avg_trade = float(np.nanmean(trade_pct_ret))

    # Trade duration stats
    dur_mins_trade = (pd.to_datetime(trade_df['exit_time']) - pd.to_datetime(trade_df['entry_time'])).dt.total_seconds() / 60.0
    max_trade_dur_mins = float(dur_mins_trade.max())
    avg_trade_dur_mins = float(dur_mins_trade.mean())

    # Profit factor
    gross_profit = float(trade_df.loc[trade_df['profit_ac'] > 0, 'profit_ac'].sum())
    gross_loss = float(trade_df.loc[trade_df['profit_ac'] < 0, 'profit_ac'].sum())
    profit_factor = gross_profit / abs(gross_loss) if gross_loss != 0 else float('inf')

    # Long/Short stats
    long_trades = trade_df[trade_df['side'] == "BUY"]
    short_trades = trade_df[trade_df['side'] == "SELL"]

    if len(long_trades) > 0:
        long_winrate = (long_trades['profit_ac'] > 0).mean() * 100.0
        long_profit = long_trades['profit_ac'].sum()

    else:
        long_winrate, long_profit = 0.0, 0.0

    if len(short_trades) > 0:
        short_winrate = (short_trades['profit_ac'] > 0).mean() * 100.0
        short_profit = short_trades['profit_ac'].sum()
    
    else:
        short_winrate, short_profit = 0.0, 0.0
    
    # Conseutive wins/losses
    profit_ac_value = trade_df['profit_ac'].values
    wins = (profit_ac_value > 0).astype(int)
    losses = (profit_ac_value < 0).astype(int)

    def longest_streak(mask, profit_array):
        max_count, max_sum, cur_count, cur_sum = 0, 0.0, 0, 0.0
        for i in range(len(profit_array)):
            if mask[i] == 1:
                cur_count += 1
                cur_sum += profit_array[i]

                if cur_count > max_count:
                    max_count = cur_count
                    max_sum = cur_sum
            else:
                cur_count, cur_sum = 0, 0.0 # reset value to 0
        
        return max_count, max_sum
    
    win_streak, win_strak_profit = longest_streak(wins, profit_ac_value)
    loss_streak, loss_streak_loss = longest_streak(losses, profit_ac_value)


    report = {
        # --- Basic info ---
        "Start date": pd.to_datetime(start),
        "End date": pd.to_datetime(end),
        "Duration (days)": duration_days,
        "Trades": int(n),

        # --- Return metrics ---
        "Equity Final ($)": round(balance_final, 2),
        "Equity Peak ($)": round(balance_peak, 2),
        "Return (%)": round(total_return, 2),
        "Return (annual - %)": round(annualized_return, 2),
        "Return (monthly - %)": round(monthly_return, 2),
        "CAGR (%)": round(annualized_return, 2),   # tương đương CAGR

        # --- Risk metrics ---
        "Sharpe ratio": round(sharpe, 3),
        "Sortino ratio": "No downside returns" if np.isinf(sortino) else round(sortino, 3),
        "Max DD (%)": round(dd_stats['max_dd_pct'], 2),
        "Avg DD (%)": round(dd_stats['avg_dd_pct'], 2),
        "Max DD Duration (days)": int(dd_stats['max_dd_duration_days']),
        "Avg DD Duration (days)": round(dd_stats['avg_dd_duration_days'], 1),

        # --- Trade-level metrics ---
        "Win rate (%)": round(winrate, 2),
        "Best trade (%)": round(best_trade, 2),
        "Worst trade (%)": round(worst_trade, 2),
        "Avg trade (%)": round(avg_trade, 2),
        "Max trade duration (mins)": round(max_trade_dur_mins, 2),
        "Avg trade duration (mins)": round(avg_trade_dur_mins, 2),
        "Profit factor": 0.0 if np.isinf(profit_factor) else round(profit_factor, 3),

        # --- Long/Short stats ---
        "Long trades winrate (%)": round(long_winrate, 2),
        "Long trades profit ($)": round(long_profit, 2),
        "Short trades winrate (%)": round(short_winrate, 2),
        "Short trades profit ($)": round(short_profit, 2),

        # --- Consecutive streaks ---
        "Consecutive wins (count)": int(win_streak),
        "Consecutive profit ($)": round(win_strak_profit, 2),
        "Consecutive losses (count)": int(loss_streak),
        "Consecutive losses ($)": round(loss_streak_loss, 2)
    }

    # Return report df, balance series and daily balance series for plotting
    report_df = pd.DataFrame([report])
    
    return report_df, balance_series, balance_daily

In [61]:
btc_report2, btc_balance, btc_daily = performance_report(btc_h1_signal_v2, trade_results_v2)
print(btc_report2.T)

                                              0
Start date                  2019-12-31 00:00:00
End date                    2025-10-29 00:00:00
Duration (days)                            2129
Trades                                      151
Equity Final ($)                       22672.84
Equity Peak ($)                        23076.07
Return (%)                               126.73
Return (annual - %)                       15.08
Return (monthly - %)                       1.18
CAGR (%)                                  15.08
Sharpe ratio                              0.634
Sortino ratio                             1.156
Max DD (%)                               -11.59
Avg DD (%)                                -5.05
Max DD Duration (days)                     2873
Avg DD Duration (days)                    725.0
Win rate (%)                              35.76
Best trade (%)                            23.28
Worst trade (%)                           -2.55
Avg trade (%)                           

In [63]:
btc_h1_signal_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12774 entries, 0 to 12773
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   time           12774 non-null  datetime64[ns]
 1   tick_volume    12774 non-null  uint64        
 2   spread         12774 non-null  int32         
 3   real_volume    12774 non-null  uint64        
 4   real_spread    12774 non-null  float64       
 5   bid_o          12774 non-null  float64       
 6   ask_o          12774 non-null  float64       
 7   bid_h          12774 non-null  float64       
 8   ask_h          12774 non-null  float64       
 9   bid_l          12774 non-null  float64       
 10  ask_l          12774 non-null  float64       
 11  bid_c          12774 non-null  float64       
 12  ask_c          12774 non-null  float64       
 13  donchian_high  12725 non-null  float64       
 14  donchian_low   12725 non-null  float64       
 15  signal         1277